In [16]:
from __future__ import division
from  imageio import imwrite 
%pylab inline
import numpy as np
import scipy as sp
import scipy.fftpack as ft

size_R_patch = 10
half_taille_patch = 3
Dim_PCA = np.round(0.5*half_taille_patch*half_taille_patch)
COEF_k = 0.4
# load an image
import imageio as imio
colored_image = imio.imread('toit.PNG')
n = colored_image.shape[0] - 1
n = 128
image = np.sum(colored_image * [0.21, 0.72 ,0.07],axis=-1)
cropped = image
image0 = np.copy(image)
def snr(x, y): # signal noise ratio
    s =  np.linalg.norm(x - y)
    if s == 0:
        return "Equal inputs"
    return 20 * np.log10(np.linalg.norm(x) / s)

def clamp0(x):
    temp = 255 * (x > 255) + x * (x < 255) 
    return temp * (x > 0)

def clamp(x, a, b):
    temp = b *(x > b) + x * (x < b)  
    result = temp * (temp > a) + a * (x < a)
    return result

def GaussianNoise(image, sigma):
    return clamp0(image + sigma * np.random.randn(image.shape[0], image.shape[1]))

def SaltNoise(image, sigma):
    return clamp0(image + sigma * (np.random.randn(image.shape[0], image.shape[1]) > 0))
sigma = 20.0
noisy_image = clamp0(image + sigma * np.random.randn(image.shape[0], image.shape[1]))
noisy_salt = clamp0(image + SaltNoise(image, 30))

#taille_patch = 2 * half_taille_patch + 1
snr_result=[0]*10
count = 0
half_taille_patch = 0
for i in range (0,10):
    half_taille_patch = half_taille_patch+1
    taille_patch = 2 * half_taille_patch + 1
    Dim_PCA = np.round(0.5*taille_patch*taille_patch)
    print(taille_patch)
    print(Dim_PCA)
    [X, Y, dX, dY] = np.meshgrid(np.arange(1, n + 1), np.arange(1, n + 1), np.arange(-half_taille_patch, half_taille_patch + 1), np.arange(-half_taille_patch, half_taille_patch + 1))
    X = X + dX # x coordinate
    Y = Y + dY # y coordinate
    # hereafter we deal with the boundaries.
    X[X < 1] = 2 - X[X < 1] #miroir sur X[i<0]
    Y[Y < 1] = 2 - Y[Y < 1] #miroir sur Y[i<10]
    X[X > n] = 2 * n - X[X > n] #miroir sur X[i>n]
    Y[Y > n] = 2 * n - Y[Y > n] #miroir sur X[i>n]
    I = (X-1) + (Y-1)*n
    for i in range(n//half_taille_patch): # // division entière
        for j in range(n//half_taille_patch):
            I[i,j] = np.transpose(I[i,j])
            #print(I[i,j],"\n")
    patch = lambda f: np.ravel(f)[I] #ravel: concatenation
    P = patch(noisy_image)
    #P = patch(image )
    # on applique la décomposition par patch I à l'image qui est d'abord concaténé.
    nb_x = image.shape[0]
    nb_y = image.shape[1]

    dimension = int(Dim_PCA)
    # a reshape function
    resh = lambda P: np.transpose((np.reshape(P, (n*n,taille_patch**2), order="F")))
    # remove mean is a preparation for PCA.
    remove_mean = lambda Q: Q - np.tile(np.mean(Q,0),(taille_patch**2,1))
    # np.tile :Construct an array by repeating A the number of times given by reps.
    # np.mean(Q,axis=0) axis==0 -> a = np.array([[1, 2], [3, 4]]) -> array([2., 3.])
    #                   axis==1 -> array([1.5, 3.5])
    P1 = remove_mean(resh(P)) # P = patch(image) 
    C = np.dot(P1,np.transpose(P1))
    [D,V] = np.linalg.eig(C)
    # the array D is the list of eigenvalues and the arry V is the corresponding eigenvectors.
    # We then sort the values in decreasing order and reindex the corresponding eigenvectors.
    D = np.sort(D)[::-1] #eingenvalues sorted
    I = np.argsort(D)[::-1] # argosort renvoie la position des egeinvalue sorted
    V = V[I,:] # renvoie les eigein vecteur en fonction de la valeur décroissante des eigeinvalue
    # L = range(10)
    #L[::-1] -> [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
    iresh = lambda Q: np.reshape(np.transpose(Q),(n,n,dimension),order="F")
    H = iresh(np.dot(np.transpose(V[: ,:dimension]),remove_mean(resh(P)))) #pscalair V et P reshape
    f = size_R_patch
    selection = lambda i: np.array((clamp(np.arange(i[0]-f,i[0] + f + 1), 0, n-1), clamp(np.arange(i[1]-f,i[1] + f + 1), 0, n-1)))
    # on vient regarder non plus toute l'image mais une range de -f à f dans laquel on regarde les patchs
    # sinon sur toute l'image la complexité est trop importante
    def distance_0(i,sel): 
        H1 = (H[sel[0],:,:])
        H2 = (H1[:,sel[1],:])
        return np.sum((H2 - np.tile(H[i[0],i[1],:],(len(sel[0]),len(sel[1]),1)))**2,2)/(taille_patch**2)

    #   b = np.array([[1, 2], [3, 4]])
    #   np.tile(b, 2)
    #   array([[1, 2, 1, 2],
    #          [3, 4, 3, 4]])

    normalize = lambda K: K/np.sum(K)
    distance = lambda i: distance_0(i, selection(i))
    kernel = lambda i, tau: normalize(np.exp(-distance(i)/ (2*tau**2)))
    tau = 10
    i = [85,40]
    D = distance(i)
    K = kernel(i, tau)
    def NLval_0(K,sel): 
        f_temp = noisy_image[sel[0],:]
        return np.sum(K*f_temp[:, sel[1]]) #mutltiplication kernel * noisy image

    NLval = lambda i, tau: NLval_0(kernel(i, tau), selection(i))
    [Y, X] = np.meshgrid(np.arange(0,n),np.arange(0,n)) #rappel n = 128 taille de l'image
    #print(X) #0- 128 sur 128 collone / Y 0-128 sur 128 ligne
    #meshgrid permet de construire des matrices ou tableau 2D a partir de range
    def arrayfun(img,X,Y):
        n = len(X)
        p = len(Y)
        R = np.zeros([n,p])
        for k in range(n):
            for l in range(p):
                R[k,l] = img(k,l)
        return R

    NLmeans = lambda tau: arrayfun(lambda i1, i2: NLval([i1,i2], tau), X, Y)
    tau=7
    imgdenoised = NLmeans(tau)
    snr_= snr(image,imgdenoised)
    snr_result[count]=snr_
    count = count + 1
    print("snr(image,imgdenoised) (i =",i,"): " ,snr_)
    

print("===========================================")
print(snr_result)

Populating the interactive namespace from numpy and matplotlib
3
4.0
snr(image,imgdenoised) (i = [85, 40] ):  16.158718446134152
5
12.0
snr(image,imgdenoised) (i = [85, 40] ):  21.018707285848365
7
24.0
snr(image,imgdenoised) (i = [85, 40] ):  21.695953088534047
9
40.0
snr(image,imgdenoised) (i = [85, 40] ):  21.46268979045054
11
60.0
snr(image,imgdenoised) (i = [85, 40] ):  21.085746381790674
13
84.0
snr(image,imgdenoised) (i = [85, 40] ):  20.824141727677585
15
112.0
snr(image,imgdenoised) (i = [85, 40] ):  20.629244109616973
17
144.0
snr(image,imgdenoised) (i = [85, 40] ):  20.47402876059803
19
180.0
snr(image,imgdenoised) (i = [85, 40] ):  20.32350091267268
21
220.0
snr(image,imgdenoised) (i = [85, 40] ):  20.222451146755365
[16.158718446134152, 21.018707285848365, 21.695953088534047, 21.46268979045054, 21.085746381790674, 20.824141727677585, 20.629244109616973, 20.47402876059803, 20.32350091267268, 20.222451146755365]
